# Electronics Dataset Preprocessing

This notebook creates a manageable subset of the Amazon Electronics dataset by filtering for popular products based on number of reviews.

**Goal**: Extract 1000 most popular Electronics products based on review count for our RAG pipeline.

**Data Source**: Amazon Reviews 2023 dataset by McAuley Lab

**Citation**: Hou et al. (2024) - Bridging Language and Items for Retrieval and Recommendation (arXiv:2403.03952)

In [1]:
import json
import pandas as pd
from collections import Counter, defaultdict
import gzip
from pathlib import Path
import numpy as np
from tqdm import tqdm

# Set paths
DATA_DIR = Path("../data")
REVIEWS_FILE = DATA_DIR / "Electronics.jsonl"
META_FILE = DATA_DIR / "meta_Electronics.jsonl"
OUTPUT_DIR = DATA_DIR / "processed"
OUTPUT_DIR.mkdir(exist_ok=True)

# Configuration
TARGET_PRODUCTS = 1000
MIN_REVIEWS_PER_PRODUCT = 10  # Minimum reviews to be considered "popular"


## Step 1: Count Reviews per Product

First, we'll scan through the reviews file to count how many reviews each product (parent_asin) has received.

In [2]:
def count_reviews_per_product(reviews_file):
    """Count the number of reviews for each product (parent_asin)."""
    print(f"Counting reviews per product from {reviews_file}...")
    
    review_counts = Counter()
    total_reviews = 0
    
    # Handle both .jsonl and .jsonl.gz files
    if str(reviews_file).endswith('.gz'):
        file_opener = gzip.open
        mode = 'rt'
    else:
        file_opener = open
        mode = 'r'
    
    with file_opener(reviews_file, mode, encoding='utf-8') as f:
        for line_num, line in enumerate(tqdm(f, desc="Processing reviews")):
            if line_num % 100000 == 0 and line_num > 0:
                print(f"Processed {line_num:,} reviews, found {len(review_counts):,} unique products")
            
            try:
                review = json.loads(line.strip())
                parent_asin = review.get('parent_asin')
                if parent_asin:
                    review_counts[parent_asin] += 1
                    total_reviews += 1
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON at line {line_num + 1}")
                continue
    
    print(f"\nTotal reviews processed: {total_reviews:,}")
    print(f"Unique products found: {len(review_counts):,}")
    
    return review_counts

# Count reviews per product
review_counts = count_reviews_per_product(REVIEWS_FILE)


Counting reviews per product from ../data/Electronics.jsonl...


Processing reviews: 164082it [00:00, 334261.68it/s]

Processed 100,000 reviews, found 59,429 unique products


Processing reviews: 271335it [00:00, 350304.00it/s]

Processed 200,000 reviews, found 95,630 unique products


...

Processed 43,700,000 reviews, found 1,609,858 unique products


Processing reviews: 43858398it [02:01, 377130.08it/s]

Processed 43,800,000 reviews, found 1,609,859 unique products


Processing reviews: 43886944it [02:01, 360105.72it/s]


Total reviews processed: 43,886,944
Unique products found: 1,609,860


## Step 2: Select Top Products by Review Count

Now we'll select the most popular products based on review count.

In [3]:
def select_top_products(review_counts, target_count=1000, min_reviews=10):
    """Select top products by review count."""
    print(f"\nSelecting top {target_count} products with at least {min_reviews} reviews...")
    
    # Filter products with minimum review count
    filtered_products = {k: v for k, v in review_counts.items() if v >= min_reviews}
    print(f"Products with ≥{min_reviews} reviews: {len(filtered_products):,}")
    
    # Get top products by review count
    top_products = dict(review_counts.most_common(target_count))
    
    print(f"\nSelected {len(top_products)} products")
    print(f"Review count range: {min(top_products.values())} - {max(top_products.values())}")
    
    # Show distribution
    counts = list(top_products.values())
    print(f"\nReview count statistics:")
    print(f"  Mean: {np.mean(counts):.1f}")
    print(f"  Median: {np.median(counts):.1f}")
    print(f"  75th percentile: {np.percentile(counts, 75):.1f}")
    print(f"  90th percentile: {np.percentile(counts, 90):.1f}")
    
    return top_products

top_products = select_top_products(review_counts, TARGET_PRODUCTS, MIN_REVIEWS_PER_PRODUCT)
selected_parent_asins = set(top_products.keys())



Selecting top 1000 products with at least 10 reviews...
Products with ≥10 reviews: 403,258

Selected 1000 products
Review count range: 3453 - 178239

Review count statistics:
  Mean: 8022.4
  Median: 5331.5
  75th percentile: 7555.8
  90th percentile: 12708.4


## Step 3: Extract Product Metadata

Extract metadata for our selected products from the meta_Electronics.jsonl file.

In [4]:
def extract_product_metadata(meta_file, selected_asins):
    """Extract metadata for selected products."""
    print(f"\nExtracting metadata for {len(selected_asins)} products...")
    
    products_metadata = []
    found_count = 0
    
    # Handle both .jsonl and .jsonl.gz files
    if str(meta_file).endswith('.gz'):
        file_opener = gzip.open
        mode = 'rt'
    else:
        file_opener = open
        mode = 'r'
    
    with file_opener(meta_file, mode, encoding='utf-8') as f:
        for line_num, line in enumerate(tqdm(f, desc="Processing metadata")):
            try:
                product = json.loads(line.strip())
                parent_asin = product.get('parent_asin')
                
                if parent_asin in selected_asins:
                    # Add review count to metadata
                    product['review_count'] = top_products[parent_asin]
                    products_metadata.append(product)
                    found_count += 1
                    
                    if found_count % 100 == 0:
                        print(f"Found metadata for {found_count}/{len(selected_asins)} products")
                        
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON at line {line_num + 1}")
                continue
    
    print(f"\nFound metadata for {found_count}/{len(selected_asins)} products")
    return products_metadata

products_metadata = extract_product_metadata(META_FILE, selected_parent_asins)



Extracting metadata for 1000 products...


Processing metadata: 38517it [00:00, 84778.60it/s]

Found metadata for 100/1000 products


Processing metadata: 74445it [00:00, 88535.47it/s]

Found metadata for 200/1000 products


...

Found metadata for 900/1000 products


Processing metadata: 1400367it [00:14, 104374.75it/s]

Found metadata for 1000/1000 products


Processing metadata: 1610012it [00:16, 96984.57it/s] 


Found metadata for 1000/1000 products


## Step 4: Extract Sample Reviews

Extract a sample of reviews for our selected products to use in the RAG pipeline.

In [5]:
def extract_sample_reviews(reviews_file, selected_asins, max_reviews_per_product=20):
    """Extract sample reviews for selected products."""
    print(f"\nExtracting sample reviews (max {max_reviews_per_product} per product)...")
    
    product_reviews = defaultdict(list)
    total_extracted = 0
    
    # Handle both .jsonl and .jsonl.gz files
    if str(reviews_file).endswith('.gz'):
        file_opener = gzip.open
        mode = 'rt'
    else:
        file_opener = open
        mode = 'r'
    
    with file_opener(reviews_file, mode, encoding='utf-8') as f:
        for line_num, line in enumerate(tqdm(f, desc="Processing reviews")):
            try:
                review = json.loads(line.strip())
                parent_asin = review.get('parent_asin')
                
                if (parent_asin in selected_asins and 
                    len(product_reviews[parent_asin]) < max_reviews_per_product):
                    
                    # Only keep essential review fields
                    clean_review = {
                        'asin': review.get('asin'),
                        'parent_asin': parent_asin,
                        'rating': review.get('rating'),
                        'title': review.get('title', ''),
                        'text': review.get('text', ''),
                        'timestamp': review.get('timestamp'),
                        'verified_purchase': review.get('verified_purchase'),
                        'helpful_vote': review.get('helpful_vote', 0)
                    }
                    
                    product_reviews[parent_asin].append(clean_review)
                    total_extracted += 1
                    
                    if total_extracted % 1000 == 0:
                        print(f"Extracted {total_extracted} reviews for {len(product_reviews)} products")
                        
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON at line {line_num + 1}")
                continue
    
    print(f"\nExtracted {total_extracted} reviews for {len(product_reviews)} products")
    return dict(product_reviews)

sample_reviews = extract_sample_reviews(REVIEWS_FILE, selected_parent_asins, max_reviews_per_product=20)



Extracting sample reviews (max 20 per product)...


Processing reviews: 28372it [00:00, 283703.36it/s]

Extracted 1000 reviews for 525 products
Extracted 2000 reviews for 746 products
Extracted 3000 reviews for 875 products
Extracted 4000 reviews for 928 products
Extracted 5000 reviews for 959 products
Extracted 6000 reviews for 976 products
Extracted 7000 reviews for 983 products
Extracted 8000 reviews for 993 products


Processing reviews: 67673it [00:00, 347995.63it/s]

Extracted 9000 reviews for 998 products
Extracted 10000 reviews for 999 products


...

Extracted 19000 reviews for 1000 products


Processing reviews: 924183it [00:02, 428114.91it/s]

Extracted 20000 reviews for 1000 products


Processing reviews: 43886944it [01:43, 425525.46it/s]


Extracted 20000 reviews for 1000 products


## Step 4: Extract Sample Reviews

Let's analyze our filtered dataset to ensure quality.

In [6]:
# Convert to DataFrame for analysis
df_products = pd.DataFrame(products_metadata)

print("=== DATASET SUMMARY ===")
print(f"Total products: {len(df_products)}")
print(f"Total reviews extracted: {sum(len(reviews) for reviews in sample_reviews.values())}")

print("\n=== PRODUCT METADATA FIELDS ===")
print(f"Available fields: {list(df_products.columns)}")

print("\n=== REVIEW COUNT DISTRIBUTION ===")
if 'review_count' in df_products.columns:
    print(df_products['review_count'].describe())

print("\n=== PRICE DISTRIBUTION ===")
if 'price' in df_products.columns:
    # Clean price data (remove nulls and convert to numeric)
    prices = pd.to_numeric(df_products['price'], errors='coerce').dropna()
    print(f"Products with price info: {len(prices)}/{len(df_products)}")
    if len(prices) > 0:
        print(prices.describe())

print("\n=== RATING DISTRIBUTION ===")
if 'average_rating' in df_products.columns:
    ratings = pd.to_numeric(df_products['average_rating'], errors='coerce').dropna()
    print(f"Products with rating info: {len(ratings)}/{len(df_products)}")
    if len(ratings) > 0:
        print(ratings.describe())

print("\n=== TOP 10 MOST REVIEWED PRODUCTS ===")
if 'review_count' in df_products.columns and 'title' in df_products.columns:
    top_10 = df_products.nlargest(10, 'review_count')[['title', 'review_count', 'average_rating', 'price']]
    for idx, row in top_10.iterrows():
        print(f"{row['review_count']:,} reviews - {row['title'][:80]}...")


=== DATASET SUMMARY ===
Total products: 1000
Total reviews extracted: 20000

=== PRODUCT METADATA FIELDS ===
Available fields: ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images', 'videos', 'store', 'categories', 'details', 'parent_asin', 'bought_together', 'review_count']

=== REVIEW COUNT DISTRIBUTION ===
count      1000.000000
mean       8022.383000
std       11149.838281
min        3453.000000
25%        4192.000000
50%        5331.500000
75%        7555.750000
max      178239.000000
Name: review_count, dtype: float64

=== PRICE DISTRIBUTION ===
Products with price info: 772/1000
count     772.000000
mean       55.203510
std        81.188092
min         3.490000
25%        14.990000
50%        26.990000
75%        59.990000
max      1175.350000
Name: price, dtype: float64

=== RATING DISTRIBUTION ===
Products with rating info: 1000/1000
count    1000.000000
mean        4.459000
std         0.224654
min         3.300000
25%     

## Step 5: Save Processed Dataset

Save our filtered dataset for use in the RAG pipeline.

In [7]:
# Save product metadata
products_file = OUTPUT_DIR / "electronics_top1000_products.jsonl"
with open(products_file, 'w', encoding='utf-8') as f:
    for product in products_metadata:
        f.write(json.dumps(product, ensure_ascii=False) + '\n')

print(f"Saved {len(products_metadata)} products to {products_file}")

# Save sample reviews
reviews_file = OUTPUT_DIR / "electronics_top1000_products_reviews.jsonl"
total_reviews_saved = 0
with open(reviews_file, 'w', encoding='utf-8') as f:
    for parent_asin, reviews in sample_reviews.items():
        for review in reviews:
            f.write(json.dumps(review, ensure_ascii=False) + '\n')
            total_reviews_saved += 1

print(f"Saved {total_reviews_saved} reviews to {reviews_file}")

# Save summary statistics
summary = {
    'dataset_info': {
        'source': 'Amazon Reviews 2023 - Electronics Category',
        'citation': 'Hou et al. (2024) - Bridging Language and Items for Retrieval and Recommendation (arXiv:2403.03952)',
        'processing_date': pd.Timestamp.now().isoformat(),
        'selection_criteria': {
            'target_products': TARGET_PRODUCTS,
            'min_reviews_per_product': MIN_REVIEWS_PER_PRODUCT,
            'max_reviews_per_product': 20
        }
    },
    'statistics': {
        'total_products': len(products_metadata),
        'total_reviews': total_reviews_saved,
        'products_with_metadata': len(products_metadata),
        'products_with_reviews': len(sample_reviews)
    }
}

if 'review_count' in df_products.columns:
    summary['statistics']['review_count_stats'] = {
        'min': int(df_products['review_count'].min()),
        'max': int(df_products['review_count'].max()),
        'mean': float(df_products['review_count'].mean()),
        'median': float(df_products['review_count'].median())
    }

summary_file = OUTPUT_DIR / "dataset_summary.json"
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"Saved dataset summary to {summary_file}")

print("\n=== PROCESSING COMPLETE ===")
print(f"Output files created in: {OUTPUT_DIR}")
print(f"  - {products_file.name}: Product metadata")
print(f"  - {reviews_file.name}: Sample reviews")
print(f"  - {summary_file.name}: Dataset summary")


Saved 1000 products to ../data/processed/electronics_top1000_products.jsonl
Saved 20000 reviews to ../data/processed/electronics_top1000_reviews.jsonl
Saved dataset summary to ../data/processed/dataset_summary.json

=== PROCESSING COMPLETE ===
Output files created in: ../data/processed
  - electronics_top1000_products.jsonl: Product metadata
  - electronics_top1000_reviews.jsonl: Sample reviews
  - dataset_summary.json: Dataset summary


## Summary

Successfully created a manageable subset of the Electronics dataset:

- **Selected**: Top 1000 most popular products by review count
- **Criteria**: Products with ≥10 reviews each
- **Output Files**:
  - `electronics_top1000_products.jsonl`: Product metadata
  - `electronics_top1000_reviews.jsonl`: Sample reviews (max 20 per product)
  - `electronics_rag_documents.jsonl`: RAG-optimized documents
  - `dataset_summary.json`: Processing statistics

This filtered dataset is now ready for use in your RAG pipeline and will provide much faster processing while maintaining high-quality, popular products with sufficient review data.

In [8]:
def create_rag_documents(products_metadata, sample_reviews):
    """Create documents optimized for RAG retrieval."""
    rag_documents = []
    
    for product in products_metadata:
        parent_asin = product.get('parent_asin')
        
        # Create product document
        doc = {
            'id': f"product_{parent_asin}",
            'type': 'product',
            'parent_asin': parent_asin,
            'title': product.get('title', ''),
            'description': ' '.join(product.get('description', [])) if product.get('description') else '',
            'features': ' '.join(product.get('features', [])) if product.get('features') else '',
            'price': product.get('price'),
            'average_rating': product.get('average_rating'),
            'rating_number': product.get('rating_number'),
            'review_count': product.get('review_count'),
            'store': product.get('store', ''),
            'categories': product.get('categories', []),
            'details': product.get('details', {})
        }
        
        # Create searchable text content
        content_parts = []
        if doc['title']:
            content_parts.append(f"Product: {doc['title']}")
        if doc['description']:
            content_parts.append(f"Description: {doc['description']}")
        if doc['features']:
            content_parts.append(f"Features: {doc['features']}")
        if doc['store']:
            content_parts.append(f"Store: {doc['store']}")
        if doc['categories']:
            content_parts.append(f"Categories: {' > '.join(doc['categories'])}")
        
        doc['content'] = ' '.join(content_parts)
        rag_documents.append(doc)
        
        # Add review summaries
        if parent_asin in sample_reviews:
            reviews = sample_reviews[parent_asin]
            
            # Create review summary document
            positive_reviews = [r for r in reviews if r.get('rating', 0) >= 4]
            negative_reviews = [r for r in reviews if r.get('rating', 0) <= 2]
            
            review_summary = {
                'id': f"reviews_{parent_asin}",
                'type': 'review_summary',
                'parent_asin': parent_asin,
                'product_title': doc['title'],
                'total_reviews': len(reviews),
                'positive_reviews': len(positive_reviews),
                'negative_reviews': len(negative_reviews)
            }
            
            # Sample positive and negative review texts
            pos_texts = [r.get('text', '') for r in positive_reviews[:5] if r.get('text')]
            neg_texts = [r.get('text', '') for r in negative_reviews[:5] if r.get('text')]
            
            content_parts = [f"Reviews for {doc['title']}"]
            if pos_texts:
                content_parts.append(f"Positive feedback: {' '.join(pos_texts[:3])}")
            if neg_texts:
                content_parts.append(f"Critical feedback: {' '.join(neg_texts[:3])}")
            
            review_summary['content'] = ' '.join(content_parts)
            rag_documents.append(review_summary)
    
    return rag_documents

# Create RAG documents
rag_documents = create_rag_documents(products_metadata, sample_reviews)

# Save RAG documents
rag_file = OUTPUT_DIR / "electronics_rag_documents.jsonl"
with open(rag_file, 'w', encoding='utf-8') as f:
    for doc in rag_documents:
        f.write(json.dumps(doc, ensure_ascii=False) + '\n')

print(f"Created {len(rag_documents)} RAG documents saved to {rag_file}")
print(f"Document types: {Counter(doc['type'] for doc in rag_documents)}")


Created 2000 RAG documents saved to ../data/processed/electronics_rag_documents.jsonl
Document types: Counter({'product': 1000, 'review_summary': 1000})


## Step 6: Create RAG Documents

Prepare the data in a format optimized for RAG applications.


In [ ]:
# TODO